In [1]:
import re

import numpy as np
import pandas as pd
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import InputLayer, Dense, Embedding, LSTM, SpatialDropout1D
from keras.callbacks import EarlyStopping, LearningRateScheduler
from sklearn.model_selection import train_test_split

Using TensorFlow backend.


In [2]:
# load dataset
train_df = pd.read_csv("data/train.csv")

# define parameters
max_features = 5000
tokenizer = Tokenizer(num_words=max_features, split=' ')
tokenizer.fit_on_texts(train_df["text"].values)
X = tokenizer.texts_to_sequences(train_df["text"].values)
X = pad_sequences(X)

In [3]:
# compose LSTM network
num_class = 3
embed_dim = 256
lstm_out = 196

model = Sequential()
model.add(Embedding(max_features, embed_dim, input_length=X.shape[1]))
model.add(SpatialDropout1D(rate=0.2))
model.add(LSTM(lstm_out, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(num_class, activation="softmax"))
model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 803, 256)          1280000   
_________________________________________________________________
spatial_dropout1d_1 (Spatial (None, 803, 256)          0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 196)               355152    
_________________________________________________________________
dense_1 (Dense)              (None, 3)                 591       
Total params: 1,635,743
Trainable params: 1,635,743
Non-trainable params: 0
_________________________________________________________________
None


In [4]:
Y = pd.get_dummies(train_df["author"]).values
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.2, random_state = 42)
print(X_train.shape, Y_train.shape)
print(X_test.shape, Y_test.shape)

((15663, 803), (15663, 3))
((3916, 803), (3916, 3))


In [5]:
BATCH_SIZE = 128
LR = 0.001
DC = 0.1
EPOCH = 5

callbacks = [EarlyStopping(monitor="loss", min_delta=0.001, patience=3),
             LearningRateScheduler(lambda epoch: LR/(1 + DC*epoch))]
model.fit(X_train, Y_train, epochs=EPOCH, batch_size=BATCH_SIZE,
          validation_data=(X_test, Y_test),
          callbacks=callbacks)

Train on 15663 samples, validate on 3916 samples
Epoch 1/5
15663/15663 [==============================] - 218s - loss: 0.8035 - acc: 0.6319 - val_loss: 0.5126 - val_acc: 0.7990
Epoch 2/5
15663/15663 [==============================] - 216s - loss: 0.4083 - acc: 0.8380 - val_loss: 0.4659 - val_acc: 0.8174
Epoch 3/5
15663/15663 [==============================] - 217s - loss: 0.3085 - acc: 0.8844 - val_loss: 0.4984 - val_acc: 0.8136
Epoch 4/5
15663/15663 [==============================] - 210s - loss: 0.2455 - acc: 0.9080 - val_loss: 0.5507 - val_acc: 0.8080
Epoch 5/5
15663/15663 [==============================] - 202s - loss: 0.2163 - acc: 0.9190 - val_loss: 0.5727 - val_acc: 0.8046
